<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/PVT/parameter_database2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install neqsim
!pip install wget

# NeqSim using your own property database

In this notebook, we demonstrate how to utilize your own customized parameter database within the NeqSim thermodynamic framework. This functionality allows you to:

*   **Extend NeqSim's capabilities:** Incorporate new components and associated properties to model systems beyond the default database.
*   **Improve accuracy:** Refine existing component parameters or introduce new interaction parameters to enhance the precision of simulations.
*   **Tailor to specific needs:** Create databases tailored to your particular research or industrial applications.

## Workflow for using a custom property database:

### Step 1: Create a User-Specific Parameter Database

Create new database files based on the format of the default NeqSim database files. These files typically have a `.csv` extension, such as `COMP_user.csv` for pure component parameters and `INTER_user.csv` for binary interaction parameters.

### Step 2: Modify or Extend the Database

Add new components, modify existing parameters, and introduce new interaction parameters within the user-specific database files. Here are some examples of common modifications:

*   **Adjusting critical properties:** Modify critical temperature (Tc), critical pressure (Pc), and acentric factor (ω) for existing components.
*   **Introducing new interaction parameters:** Add binary interaction parameters for component pairs, such as those used in activity coefficient models.
*   **Modifying equation of state parameters:** Adjust parameters specific to the equation of state used in your simulations.

### Step 3: Replace the Default Database

Replace the default NeqSim database with your custom database using the `neqsim.util.database.NeqSimDataBase.replaceTable` function. Here's an example code snippet:

# Further reading
See:
https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/PVT/parameter_database.ipynb


and

https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/thermodynamics/readproperties.ipynb

and

https://github.com/equinor/neqsim-parameterfitting

# NeqSim parameter database
NeqSim reads pure and mixture component parmaters from a built in database.

The database tables used are stored in this folder:

https://github.com/equinor/neqsim/tree/master/src/main/resources/data

The two main tables are the COMP.csv file with pure component parameters and the INTER.csv file with the binary interaction parameters.

THe other tables are also important but used for various specialized calculations.

# Use your own parameter database

You can establish your own database table by storing your parameters in a file with the same format as, for example, COMP.csv, and set that table to be used by NeqSim.

In the following example, we will show how this can be done.

We first download the COMP and INTER tables from GitHub and then rename them to COMP_user and INTER_user. The tables can then be changed or extended by the user. These files should be stored on your local drive and read into neqsim every time you run the program.

After this is done, we write the tables back to NeqSim, and the new tables will be used as the basis for calculations.

In [ ]:
# we start by downloading the COMP and INTER tables

%%capture
!wget https://raw.githubusercontent.com/equinor/neqsim/master/src/main/resources/data/COMP.csv
!wget https://raw.githubusercontent.com/equinor/neqsim/master/src/main/resources/data/INTER.csv

import os

# Rename the COMP table
os.rename('COMP.csv', 'COMP_user.csv')

# Rename the INTER table
os.rename('INTER.csv', 'INTER_user.csv')

!chmod 644 /content/COMP_user.csv

Further, we will modify the tables and add components directly in the csv text file (not done here).

We will then write it back to NeqSim by overwriting the COMP or INTER database.

In [ ]:
from neqsim import jneqsim as neqsim
from neqsim.thermo import TPflash, printFrame

# Replace the default 'COMP' table in the NeqSim database with the user-defined 'COMP_user.csv' table
# located at '/content/COMP_user.csv'.
neqsim.util.database.NeqSimDataBase.replaceTable('COMP', '/content/COMP_user.csv')
# Replace the default 'INTER' table in the NeqSim database with the user-defined 'INTER_user.csv' table
# located at '/content/INTER_user.csv'.
neqsim.util.database.NeqSimDataBase.replaceTable('INTER', '/content/INTER_user.csv')

#and then use the new component and table
from neqsim.thermo import fluid

fluid1 = fluid('srk')
fluid1.setTemperature(20.0, 'C')
fluid1.setPressure(1.1, 'bara')
fluid1.addComponent('methane', 1.0)
TPflash(fluid1)
printFrame(fluid1)

| 0                    | 1         | 2          | 3   | 4   | 5   | 6               |
|:---------------------|:----------|:-----------|:----|:----|:----|:----------------|
|                      | total     | GAS        |     |     |     |                 |
| methane              | 1E0       | 1E0        |     |     |     | [mole fraction] |
|                      |           |            |     |     |     |                 |
| Density              |           | 7.25498E-1 |     |     |     | kg/m3           |
| Phase Fraction       |           | 1E0        |     |     |     | [mole fraction] |
| Molar Mass           | 1.6043E-2 | 1.6043E-2  |     |     |     | kg/mol          |
| Z factor             |           | 9.97971E-1 |     |     |     | [-]             |
| Heat Capacity (Cp)   |           | 2.22537E0  |     |     |     | kJ/kgK          |
| Heat Capacity (Cv)   |           | 1.70117E0  |     |     |     | kJ/kgK          |
| Speed of Sound       |           | 4.44916E2  |     

## Modifying Parameters Directly in Program Code

NeqSim provides the flexibility to modify component parameters directly within your program code. This approach allows for dynamic adjustments during simulations, offering greater control over the thermodynamic properties of your system.

Instead of relying on external database files, you can set parameters like critical temperature (Tc), critical pressure (Pc), and acentric factor directly on the fluid object using NeqSim's API.

See:

https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/thermodynamics/readproperties.ipynb

Also see:

https://github.com/equinor/neqsim-parameterfitting

**Benefits of In-Code Parameter Modification:**

*   **Dynamic Adjustments:** Change parameters during runtime based on simulation conditions or specific requirements.
*   **Flexibility and Control:** Fine-tune component properties without modifying external files.
*   **Enhanced Customization:** Implement custom property calculations or correlations directly within your code.

**Example:**

In [ ]:
from neqsim.thermo import fluid, TPflash, printFrame

# Create a fluid object using the SRK equation of state
fluid1 = fluid('srk')

# Add methane to the fluid with a mole fraction of 1.0
fluid1.addComponent('methane', 1.0)

# Set the fluid temperature and pressure
fluid1.setTemperature(20.0, 'C')
fluid1.setPressure(1.1, 'bara')

for phase_number in range(fluid1.getMaxNumberOfPhases()):
  # Get the component (methane) and modify its properties
  methane = fluid1.getPhase(phase_number).getComponent('methane')

  # Change critical temperature (Tc) to 195.0 K
  methane.setTC(190.56, 'K')
  # Change critical pressure (Pc) to 50.0 bara
  methane.setPC(45.992, 'bara')
  # Change acentric factor (omega) to 0.011
  methane.setAcentricFactor(0.011)
  # Change molar mass
  methane.setMolarMass(16.04, 'gr/mol')

# Perform a two-phase flash calculation on the fluid
TPflash(fluid1)

# Print the results of the flash calculation in a formatted table
printFrame(fluid1)

| 0                    | 1         | 2          | 3   | 4   | 5   | 6               |
|:---------------------|:----------|:-----------|:----|:----|:----|:----------------|
|                      | total     | GAS        |     |     |     |                 |
| methane              | 1E0       | 1E0        |     |     |     | [mole fraction] |
|                      |           |            |     |     |     |                 |
| Density              |           | 7.25364E-1 |     |     |     | kg/m3           |
| Phase Fraction       |           | 1E0        |     |     |     | [mole fraction] |
| Molar Mass           | 1.604E-2  | 1.604E-2   |     |     |     | kg/mol          |
| Z factor             |           | 9.97969E-1 |     |     |     | [-]             |
| Heat Capacity (Cp)   |           | 2.22578E0  |     |     |     | kJ/kgK          |
| Heat Capacity (Cv)   |           | 1.70149E0  |     |     |     | kJ/kgK          |
| Speed of Sound       |           | 4.44957E2  |     